### Creations Graphs

In [1]:
!pip3 install psycopg2 --user
!pip3 install sshtunnel --user

  Using cached sshtunnel-0.4.0-py2.py3-none-any.whl (24 kB)
  Using cached paramiko-2.12.0-py2.py3-none-any.whl (213 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-macosx_10_10_universal2.whl (349 kB)
  Using cached bcrypt-4.0.1-cp36-abi3-macosx_10_10_universal2.whl (473 kB)
  Using cached cryptography-38.0.3-cp36-abi3-macosx_10_10_x86_64.whl (2.8 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import psycopg2
import sshtunnel 
import pandas as pd
import psycopg2.extras


def get_data(c):
        pd.set_option("display.max.columns", None)
        #"agency", "calendar","calendar_dates", "routes", "shapes"
        list_tables=["stop_times","stops", "translations","trips","vehicleposition" ]
        #Get stats and graphs for each table
        for i in list_tables:
                c.execute(f"SELECT * FROM {i}")
                data = c.fetchall()
                df=pd.DataFrame([i.copy() for i in data])
                df2=pd.DataFrame(df.isna().sum(), columns=[ "count_missing_values"])
                df2.to_csv("./dataPreparation/stadistics/missing_values_"+i+".csv")

                df3=pd.DataFrame(df.describe())
                df3.to_csv("./dataPreparation/stadistics/describe_num_"+i+".csv")

                df4=pd.DataFrame(df.describe(include=object))
                df4.to_csv("./dataPreparation/stadistics/describe_str_"+i+".csv")

                fig=df.isna().sum().reset_index(name="missing_values").plot.bar(x='index', y='missing_values', rot=45) 
                fig.get_figure().savefig("./dataPreparation/stadistics/graph_missing_values_"+i+".png")


try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                c = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
                get_data(c)
except Exception as e:
        print(e)






### Connect with ssh 
ssh -N -L 1234:stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com:5432 ec2-user@3.83.88.154 -i /Users/lauraforerocamacho/Downloads/rds-example.cer

